In [1]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import os
import urllib.request
from urllib.request import urlopen
import numpy as np
import pandas as pd
from pandas import DataFrame
from requests.exceptions import ConnectionError

In [2]:
internal_urls = set()
external_urls = set()

In [3]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

In [4]:
#The function gives all urls
count = 0
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        global count
        count = count+1
        print(f"{count}[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
        print(external_urls)
    return urls

In [5]:
total_urls_visited = 0
def crawl(url, max_urls=30000):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except Exception as e:
        print(e)

In [6]:
#base_url = "https://umd.edu/virusinfo"
#Input
# base_url = input("Enter the URL : ")
base_url = "https://www.beryl.nyc"
internal_urls.add(base_url)

In [7]:
parsedurl = urlparse(base_url)
print("parsedurl")
print(parsedurl)
base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
print(base_url_main)
# base_url_text = base_url.split("//",1)[1]
base_url_text = parsedurl.netloc+parsedurl.path
print("base_url_text")
print(base_url_text)
# base_url_text_domain = base_url_text.split("/",1)[0]
base_url_text_domain = parsedurl.netloc
print(base_url_text_domain)

parsedurl
ParseResult(scheme='https', netloc='www.beryl.nyc', path='', params='', query='', fragment='')
https://www.beryl.nyc
base_url_text
www.beryl.nyc
www.beryl.nyc


In [8]:
crawl(base_url)
print("[+] Total External links:", len(external_urls))
#print("[+] Total Internal links:", len(internal_urls))
print("[+] Total:", len(external_urls) + len(internal_urls))

1[INT] Internal link: https://www.beryl.nyc/
set()
2[INT] Internal link: https://www.beryl.nyc/index.php/recipe-index/
set()
3[INT] Internal link: https://www.beryl.nyc/index.php/articles/
set()
4[INT] Internal link: https://www.beryl.nyc/index.php/artists/
set()
5[INT] Internal link: https://www.beryl.nyc/index.php/contact-me/
set()
6[INT] Internal link: https://www.beryl.nyc/index.php/about-2/
set()
7[INT] Internal link: https://www.beryl.nyc/index.php/faq/
set()
8[INT] Internal link: https://www.beryl.nyc/index.php/search
set()
9[INT] Internal link: https://www.beryl.nyc/index.php/2024/06/02/7-things-to-order-off-a-persian-menu/
{'https://youtu.be/YkDsn-hfxwg'}
10[INT] Internal link: https://www.beryl.nyc/index.php/2023/11/14/whats-in-your-fridge/
{'https://youtu.be/YkDsn-hfxwg'}
11[INT] Internal link: https://www.beryl.nyc/index.php/2024/01/25/jia-yi-liang-mian/
{'https://youtu.be/YkDsn-hfxwg'}
12[INT] Internal link: https://www.beryl.nyc/index.php/2024/09/05/apple-dumplings/
{'htt

In [12]:
#Converting List to Column
df = DataFrame(internal_urls,columns=['URL'])
print(df.head(n=10).to_string(index=False))
df.to_csv('filename1.csv')

                                                                            URL
              https://www.beryl.nyc/index.php/wprm_print/mach-er-kata-chorchori
                        https://www.beryl.nyc/index.php/wprm_print/schmorgurken
                            https://www.beryl.nyc/index.php/masiel-soler-urzua/
https://www.beryl.nyc/index.php/2023/05/16/guisado-de-spam-con-platano-maduros/
                             https://www.beryl.nyc/index.php/2024/07/05/rourou/
                 https://www.beryl.nyc/index.php/wprm_print/watermelon-gazpacho
                                     https://www.beryl.nyc/index.php/tag/snack/
                              https://www.beryl.nyc/index.php/wprm_print/cevapi
                      https://www.beryl.nyc/index.php/2023/05/16/seca-de-pollo/
                                    https://www.beryl.nyc/index.php/tag/africa/


In [13]:
dfrecipe = df[df['URL'].str.contains('wprm_print')]
print(dfrecipe.head(n=10).to_string(index=False))

                                                              URL
https://www.beryl.nyc/index.php/wprm_print/mach-er-kata-chorchori
          https://www.beryl.nyc/index.php/wprm_print/schmorgurken
   https://www.beryl.nyc/index.php/wprm_print/watermelon-gazpacho
                https://www.beryl.nyc/index.php/wprm_print/cevapi
            https://www.beryl.nyc/index.php/wprm_print/flat-white
 https://www.beryl.nyc/index.php/wprm_print/seattle-style-hot-dog
       https://www.beryl.nyc/index.php/wprm_print/salatat-blankit
             https://www.beryl.nyc/index.php/wprm_print/pav-bhaji
     https://www.beryl.nyc/index.php/wprm_print/empanada-de-verde
            https://www.beryl.nyc/index.php/wprm_print/amrakhanda


In [14]:
dfrecipe.to_csv('filename.csv')